In [1]:
print("hello")

hello


In [2]:
import langchain
langchain.__version__

'0.3.19'

In [1]:
import os 
from dotenv import load_dotenv 
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [2]:
from langchain.agents import initialize_agent,Tool
from langchain_groq import ChatGroq

llm=ChatGroq(temperature=0.1)

In [5]:
#old langchain agent me  hum bas completion model use  karte hai coz ye chat completion ke sath comfortable nahi hai

In [6]:
#!pip install  duckduckgo-search

##DuckDuckGo search

In [12]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo Search",
    func= DuckDuckGoSearchRun().run,
    description= "Use this tool to perform web searches using DuckDuckGo"
)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

#pip install -U httpcore duckduckgo-search #for http2 error
response = agent.run("What are the latest top-rated movies in only 2025 hollywood?")

print(response)



> Entering new AgentExecutor chain...
To answer this question, I need to find out the latest top-rated movies that are scheduled to be released in Hollywood in the year 2025. I will use the DuckDuckGo Search tool to find this information.

Action: DuckDuckGo Search
Action Input: "top rated movies 2025 hollywood"
Observation: Mark your calendars, because 2025 is set to be a big year for moviegoers. While 2024 celebrated the experiential (think Emilia Pérez , The Substance , and more) this year is all about embracing ... Outside of a few box-office smashes, 2024 was a relatively quiet year for movies, full of fascinating breakouts and leftfield successes, but few major events. But 2025 is shaping up a bit differently. Better Man (2024). Release Date: January 10, 2025 (after limited release on 12/25/24) Director: Michael Gracey Starring: Jonno Davies, Steve Pemberton, Damon Herriman, Kate Mulvany, Alison Steadman Micheal Gracey, director of The Greatest Showman, takes the reins in a bio

In [13]:
##wikipedia search

In [4]:
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached beautifulsoup4-4.13.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.3-py3-none-any.whl (186 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [5]:

from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

search_tool = Tool(
    name = "Wikipedia Search",
    func= WikipediaAPIWrapper().run,
    description= "Use this tool to perform web searches on  Wikipedia"
)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

#pip install -U httpcore duckduckgo-search #for http2 error
response = agent.run("who is elon musk?")

print(response)

C:\Users\panka\AppData\Local\Temp\ipykernel_12620\2569276379.py:11: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)
C:\Users\panka\AppData\Local\Temp\ipykernel_12620\2569276379.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` i



> Entering new AgentExecutor chain...
I need to search for more information about Elon Musk on Wikipedia.

Action: Wikipedia Search
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk (; born June 28, 1971) is a businessman known for his key roles in Tesla, Inc., SpaceX, and X, formerly Twitter (which he has owned since 2022). Since 2025, he has been a senior advisor to President Donald Trump and head of the Department of Government Efficiency. Musk is the wealthiest person in the world; as of February 2025, Forbes estimates his net worth to be US$397 billion. 
Musk was born to an affluent South African family in Pretoria before immigrating to Canada, acquiring Canadian citizenship via his mother. He moved to California in 1995 to attend Stanford University, and with his brother Kimbal co-founded the software company Zip2, which was acquired by Compaq in 1999. That same year, Musk co-founded X.com, a direct bank, that later formed PayPal. In 2002, Musk acqui

##SQLite database

In [11]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("chatbot.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [12]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 


# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///chatbot.db")

# Step 3: Initialize the LLM
llm = ChatGroq(temperature=0.1)

# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
To answer this question, I need to query the database table that contains information about the products in stock. First, I should check if such a table exists.

Action: sql_db_list_tables
Action Input: ""

Observation: inventory
Thought:I see that there is a table called "inventory" in the database. Now, I need to check its schema to see if it has the necessary fields to answer the question.

Action: sql_db_schema
Action Input: inventory

Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought:The schema of the "inventory" table shows that it has the necessary fields: "id", "product\_name", and "stock\_quantity". I can see from the sample rows that there is a product named "Product A" with a stock quantity of 150. Therefore, the final answer to

In [13]:
##Archaiv Querry Run

In [19]:
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [20]:
#!pip install arxiv

In [21]:
arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

llm = ChatGroq(temperature=0.1)

tools = [arxiv_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query = "Search for research papers on Object Detection in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
To answer this question, I need to search for research papers on Object Detection published in 2024. I will use the Arxiv Search tool for this task.

Action: Arxiv Search
Action Input: {'query': 'Object Detection', 'year': 2024}
Observation: No good Arxiv Result was found
Thought:It seems that there are no research papers on Object Detection published in 2024 yet. I will broaden my search to include the past few years.

Action: Arxiv Search
Action Input: {'query': 'Object Detection', 'year_range': [2022, 2023]}
Observation: No good Arxiv Result was found
Thought:It's quite surprising that there are no recent research papers on Object Detection, given that it's a popular and rapidly evolving field in computer vision. I will widen the search range to include the past 5 years.

Action: Arxiv Search
Action Input: {'query': 'Object Detection', 'year_range': [2019, 2023]}
Observation: No good Arxiv Result was found
Thought:It's quite unexpected to not 

In [22]:
##File Agents

In [23]:
from langchain.agents import initialize_agent, Tool
from langchain_community.tools.file_management import ReadFileTool

In [29]:
read_file_tool = Tool(
    name="Read File",
    func=ReadFileTool().run,
    description="Use this tool to read the contents of a file."
)

llm = ChatGroq(temperature=0.1)

tools = [read_file_tool]

agent = initialize_agent(tools,llm,agent="zero-shot-react-description",verbose=True
)
file_name = "sample.txt"
response = agent.run(f"Read the contents of the file named {file_name}")
print("Response from Agent:")   
print(response)



> Entering new AgentExecutor chain...
To answer this question, I need to use the "Read File" tool to read the contents of the file named "sample.txt".

Action: Read File
Action Input: 'sample.txt'
Observation: Error: no such file or directory: 'sample.txt'
Thought:It seems that the file "sample.txt" does not exist in the current working directory. I will report this observation to the user.

Final Answer: I'm unable to read the contents of the file named "sample.txt" because it doesn't exist in the current working directory.

> Finished chain.
Response from Agent:
I'm unable to read the contents of the file named "sample.txt" because it doesn't exist in the current working directory.


In [30]:
%pwd

'c:\\Users\\panka\\OneDrive\\Desktop\\Ai_agents\\Ai_Agents\\4.23_Langgraph_multi_agent'